## EDA and Modelling to predict Match Result 

### Step 1: Reading and Understanding the Data

Let us first import all libraries and dependencies

In [1]:
# import all libraries and dependencies for dataframe
from pyforest import *
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta

# import all libraries and dependencies for data visualization
pd.options.display.float_format='{:.4f}'.format
plt.rcParams['figure.figsize'] = [8,8]
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1) 
sns.set(style='darkgrid')
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker

# import all libraries and dependencies for machine learning
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.base import TransformerMixin

In [13]:
# Local file path. Please change the file path accordingly

path = 'C:\\Users\\I330405\\Documents\\HACKATHON\\'

In [14]:
# Reading the Team_leaderboard file on which Analysis needs to be done

file = path + '2019_Pro_Kabaddi.xlsx'
df_mr = pd.read_excel(file)

df_mr.head(100)

,Week,Team_1,Team_2,Team 1_Score,Team 2_Score,Winner,MarginScore
0,Week 1,Telugu Titans,U Mumba,25,31,U Mumba,-6
1,Week 1,Bengaluru Bulls,Patna Pirates,34,32,Bengaluru Bulls,2
2,Week 1,Bengaluru Bulls,Fortune Giants,24,42,Fortune Giants,-18
3,Week 1,Telugu Titans,Tamil Thalaivas,26,39,Tamil Thalaivas,-13
4,Week 1,U Mumba,Pink Panthers,23,42,Pink Panthers,-19
...,...,...,...,...,...,...,...
95,Week 9,Bengal Warriors,Haryana Steelers,48,36,Bengal Warriors,12
96,Week 9,Telugu Titans,Patna Pirates,42,42,Patna Pirates,0
97,Week 9,Puneri Paltan,Bengaluru Bulls,42,38,Puneri Paltan,4
98,Week 10,Pink Panthers,Fortune Giants,28,28,Fortune Giants,0


In [15]:
df_mr = df_mr.drop(['Week','Team 1_Score','Team 2_Score','MarginScore'],axis=1)
df_mr.head() 

,Team_1,Team_2,Winner
0,Telugu Titans,U Mumba,U Mumba
1,Bengaluru Bulls,Patna Pirates,Bengaluru Bulls
2,Bengaluru Bulls,Fortune Giants,Fortune Giants
3,Telugu Titans,Tamil Thalaivas,Tamil Thalaivas
4,U Mumba,Pink Panthers,Pink Panthers


In [16]:
#Building the model

df_mr = df_mr.reset_index(drop=True)
df_mr.loc[df_mr.Winner == df_mr.Team_1,'winning_team']=1
df_mr.loc[df_mr.Winner == df_mr.Team_2, 'winning_team']=2
df_mr = df_mr.drop(['winning_team'], axis=1)

df_mr.head()

,Team_1,Team_2,Winner
0,Telugu Titans,U Mumba,U Mumba
1,Bengaluru Bulls,Patna Pirates,Bengaluru Bulls
2,Bengaluru Bulls,Fortune Giants,Fortune Giants
3,Telugu Titans,Tamil Thalaivas,Tamil Thalaivas
4,U Mumba,Pink Panthers,Pink Panthers


### Step 2: Splitting the Data into Training and Testing Sets

As you know, the first basic step for regression is performing a train-test split.

In [17]:
#convert team-1 and team-2 from categorical variables to continous inputs 
# Get dummy variables
final = pd.get_dummies(df_mr, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

# Separate X and y sets
X = final.drop(['Winner'], axis=1)
y = final["Winner"]


# Separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

### Step 3: Building a ML model

#### Logistic Regression

In [18]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
Y_pred = logreg.predict(X_test)

score = logreg.score(X_train, y_train)
score2 = logreg.score(X_test, y_test)

print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.863
Test set accuracy:  0.714


#### Decision Tree

In [19]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
Y_pred = decision_tree.predict(X_test)


score = decision_tree.score(X_train, y_train)
score2 = decision_tree.score(X_test, y_test)

print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.912
Test set accuracy:  0.600


#### Random Forest

In [20]:
rf = RandomForestClassifier(n_estimators=100, max_depth=20,
                              random_state=100)
rf.fit(X_train, y_train) 


score = rf.score(X_train, y_train)
score2 = rf.score(X_test, y_test)


print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.912
Test set accuracy:  0.686


In [21]:
#adding WINS
#the team which is positioned higher on the WIN table will be considered "favourite" for the match
#and therefore, will be positioned under the "Team_1" column

path = 'C:\\Users\\I330405\\Documents\\HACKATHON\\'
file = path + '2019_Pro_Kabaddi.xlsx'
ranking = pd.read_excel(file,'Points Table_2019')

fixtures = pd.read_csv('C:\\Users\\I330405\\Documents\\HACKATHON\\kabbadi_fixtures.csv')

# List for storing the group stage games
pred_set = []
fixtures.head(135)
ranking.head(12)

,Team,Played,WINS,LOSSES,DRAWS,Score Diff,Points
0,Dabang Delhi,20,15,3,2,92,82
1,Bengal Warriors,20,13,4,3,86,78
2,Haryana Steelers,19,12,6,1,24,65
3,U Mumba,19,10,8,1,33,59
4,Bengaluru Bulls,19,10,8,1,12,58
5,UP Yoddha,18,10,6,2,-19,58
6,Pink Panthers,20,8,10,2,-18,52
7,Puneri Paltan,20,6,11,3,-71,42
8,Patna Pirates,19,6,12,1,-1,40
9,Fortune Giants,20,6,12,2,14,45


In [22]:
# Create new columns with wins of each team
fixtures.insert(1, 'First_Position', fixtures['Team_1'].map(ranking.set_index('Team')['WINS']))
fixtures.insert(2, 'Second_Position', fixtures['Team_2'].map(ranking.set_index('Team')['WINS']))


In [23]:
# We only need the group stage games, so we have to slice the dataset
fixtures = fixtures.iloc[:131, :]
fixtures.head(10)

,Team_1,First_Position,Second_Position,Team_2,Winner
0,Telugu Titans,5.0000,10.0000,U Mumba,nan
1,Bengaluru Bulls,10.0000,6.0000,Patna Pirates,nan
2,Bengaluru Bulls,10.0000,6.0000,Fortune Giants,nan
3,Telugu Titans,5.0000,3.0000,Tamil Thalaivas,nan
4,U Mumba,10.0000,8.0000,Pink Panthers,nan
5,Puneri Paltan,6.0000,12.0000,Haryana Steelers,nan
6,UP Yoddha,10.0000,13.0000,Bengal Warriors,nan
7,Telugu Titans,5.0000,15.0000,Dabang Delhi,nan
8,UP Yoddha,10.0000,6.0000,Fortune Giants,nan
9,Telugu Titans,5.0000,6.0000,Patna Pirates,nan


In [24]:
# Loop to add teams to new prediction dataset based on the wins of each team
for index, row in fixtures.iterrows():
    if row['First_Position'] > row['Second_Position']:
        pred_set.append({'Team_1': row['Team_1'], 'Team_2': row['Team_2'], 'winning_team': None})
    else:
        pred_set.append({'Team_1': row['Team_2'], 'Team_2': row['Team_1'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set

In [25]:
# Get dummy variables and drop winning_team column
pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

# Add missing columns compared to the model's training dataset
missing_cols = set(final.columns) - set(pred_set.columns)

for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[final.columns]


pred_set = pred_set.drop(['Winner'], axis=1)
pred_set.head()

,Team_1_Bengal Warriors,Team_1_Bengaluru Bulls,Team_1_Dabang Delhi,Team_1_Fortune Giants,Team_1_Haryana Steelers,Team_1_Patna Pirates,Team_1_Pink Panthers,Team_1_Puneri Paltan,Team_1_Tamil Thalaivas,Team_1_Telugu Titans,Team_1_U Mumba,Team_1_UP Yoddha,Team_2_Bengal Warriors,Team_2_Bengaluru Bulls,Team_2_Dabang Delhi,Team_2_Fortune Giants,Team_2_Haryana Steelers,Team_2_Patna Pirates,Team_2_Pink Panthers,Team_2_Puneri Paltan,Team_2_Tamil Thalaivas,Team_2_Telugu Titans,Team_2_U Mumba,Team_2_UP Yoddha
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0


In [26]:
#group matches 
predictions = rf.predict(pred_set)
for i in range(fixtures.shape[0]):
    print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
    if predictions[i] == 1:
        print("Winner: " + backup_pred_set.iloc[i, 1])
    
    else:
        print("Winner: " + backup_pred_set.iloc[i, 0])
    print("")

Telugu Titans and U Mumba
Winner: U Mumba

Patna Pirates and Bengaluru Bulls
Winner: Bengaluru Bulls

Fortune Giants and Bengaluru Bulls
Winner: Bengaluru Bulls

Tamil Thalaivas and Telugu Titans
Winner: Telugu Titans

Pink Panthers and U Mumba
Winner: U Mumba

Puneri Paltan and Haryana Steelers
Winner: Haryana Steelers

UP Yoddha and Bengal Warriors
Winner: Bengal Warriors

Telugu Titans and Dabang Delhi
Winner: Dabang Delhi

Fortune Giants and UP Yoddha
Winner: UP Yoddha

Telugu Titans and Patna Pirates
Winner: Patna Pirates

Puneri Paltan and U Mumba
Winner: U Mumba

Pink Panthers and Bengal Warriors
Winner: Bengal Warriors

Haryana Steelers and Dabang Delhi
Winner: Dabang Delhi

U Mumba and Bengaluru Bulls
Winner: Bengaluru Bulls

Tamil Thalaivas and Patna Pirates
Winner: Patna Pirates

Puneri Paltan and Bengal Warriors
Winner: Bengal Warriors

Pink Panthers and Haryana Steelers
Winner: Haryana Steelers

U Mumba and UP Yoddha
Winner: UP Yoddha

Fortune Giants and Dabang Delhi
Winne

In [27]:
def clean_and_predict(matches, ranking, final, rf):

   
    positions = []

    # Loop to retrieve each team's position according to ICC ranking
    for match in matches:
        positions.append(ranking.loc[ranking['Team'] == match[0],'WINS'].iloc[0])
        positions.append(ranking.loc[ranking['Team'] == match[1],'WINS'].iloc[0])
    
    # Creating the DataFrame for prediction
    pred_set = []

   
    i = 0
    j = 0

   
    while i < len(positions):
        dict1 = {}

        # If wins of first team is better then this team will be the 'Team_1' team, and vice-versa
        if positions[i] > positions[i + 1]:
            dict1.update({'Team_1': matches[j][0], 'Team_2': matches[j][1]})
        else:
            dict1.update({'Team_1': matches[j][1], 'Team_2': matches[j][0]})

        # Append updated dictionary to the list, that will later be converted into a DataFrame
        pred_set.append(dict1)
        i += 2
        j += 1
        
        # Convert list into DataFrame
    pred_set = pd.DataFrame(pred_set)
    backup_pred_set = pred_set

    # Get dummy variables and drop winning_team column
    pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

    # Add missing columns compared to the model's training dataset
    missing_cols2 = set(final.columns) - set(pred_set.columns)
    for c in missing_cols2:
        pred_set[c] = 0
    pred_set = pred_set[final.columns]

    pred_set = pred_set.drop(['Winner'], axis=1)

    # Predict
    predictions = rf.predict(pred_set)
    for i in range(len(pred_set)):
        print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
        if predictions[i] == 1:
            print("Winner: " + backup_pred_set.iloc[i, 1])
        else:
            print("Winner: " + backup_pred_set.iloc[i, 0])
        print("")

In [38]:
elim1 = [('Haryana Steelers', 'Bengaluru Bulls')]
elim2 = [('U Mumba', 'UP Yoddha')]

In [39]:
clean_and_predict(elim1, ranking, final, rf)

Bengaluru Bulls and Haryana Steelers
Winner: Haryana Steelers



In [40]:
clean_and_predict(elim2, ranking, final, rf)

U Mumba and UP Yoddha
Winner: UP Yoddha



In [41]:
semi1 = [('Haryana Steelers', 'Dabang Delhi')]
semi2 = [('UP Yoddha', 'Bengal Warriors')]

In [42]:
clean_and_predict(semi1, ranking, final, rf)

Haryana Steelers and Dabang Delhi
Winner: Dabang Delhi



In [43]:
clean_and_predict(semi2, ranking, final, rf)

UP Yoddha and Bengal Warriors
Winner: Bengal Warriors



In [48]:
finale = [('Dabang Delhi','Bengal Warriors')]

In [47]:
#Winner: Bengal Warriors based on head to head and super performance